### **Homework: Thực hiện Fine-Tuning với LoRA trên Embedding Models**

## Mục tiêu

Bài tập này nhằm giúp bạn nắm vững và áp dụng kỹ thuật **LoRA (Low-Rank Adaptation)** để fine-tuning một mô hình **Embedding**.

Cụ thể, bạn sẽ thực hành trên mô hình **Qwen/Qwen3-0.6B** (một phiên bản trong họ model Qwen) để tối ưu hóa hiệu suất của nó cho nhiệm vụ **Semantic Textual Similarity (STS)** — đánh giá mức độ tương đồng về ngữ nghĩa giữa hai câu.

## Bối cảnh

Các mô hình embedding được huấn luyện trước (**pre-trained**) thường có kiến thức tổng quát về ngôn ngữ nhưng chưa chắc đã tối ưu cho một tác vụ cụ thể như STS. **Fine-tuning** giúp "chuyên môn hóa" mô hình cho tác vụ đó.

Tuy nhiên, fine-tuning toàn bộ một mô hình lớn đòi hỏi tài nguyên tính toán khổng lồ. **LoRA** là một kỹ thuật hiệu quả (**parameter-efficient**) cho phép chúng ta đạt được hiệu suất cao bằng cách chỉ huấn luyện một phần rất nhỏ các tham số của mô hình, giúp tiết kiệm đáng kể thời gian và chi phí.

## Yêu cầu thực hiện (Dự kiến)

1. **Chuẩn bị Dữ liệu:** Tải và tiền xử lý bộ dữ liệu STS (ví dụ: một phần của tập STS Benchmark).
2. **Tải Mô hình:** Tải mô hình `Qwen/Qwen3-0.6B` và Tokenizer tương ứng.
3. **Áp dụng LoRA:** Tạo và áp dụng cấu hình `LoraConfig` cho mô hình.
4. **Huấn luyện:** Thiết lập `Trainer` và bắt đầu quá trình fine-tuning LoRA.
5. **Lưu model:** Lưu lại model và tokenizer

### **Bước 1: Chuẩn bị môi trường**

Cài đặt các thư viện cần thiết: transformers, datasets, peft (cho LoRA), accelerate, scikit-learn (để đánh giá),..

In [1]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

In [2]:
!pip install transformers accelerate peft datasets trl torch torchvision torchaudio sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using device:", device)

Using device: cuda


### **Bước 2: Tải và chuẩn bị dữ liệu**

Ở bước này, chúng ta sẽ tiến hành đọc và xử lý dữ liệu thô từ tệp sts-train.csv. Mục tiêu là chuyển đổi dữ liệu thành một định dạng có cấu trúc, phù hợp để đưa vào mô hình ngôn ngữ cho việc huấn luyện.

Quá trình này bao gồm các công việc chính sau:

- Đọc dữ liệu: Sử dụng thư viện pandas để tải dữ liệu từ tệp CSV vào một DataFrame, giúp dễ dàng truy cập và thao tác.

- Tái cấu trúc dữ liệu: Khởi tạo một cấu trúc dữ liệu mới (ở đây là một dictionary) để lưu trữ các cặp câu (sentence1, sentence2) và nhãn (label) tương ứng của chúng.
- Trích xuất và điền dữ liệu: Lặp qua từng dòng của DataFrame, lấy ra giá trị của các cột cần thiết và đưa chúng vào dictionary đã tạo.

In [4]:
# Dataset preparation
import pandas as pd
from datasets import load_dataset, Dataset

# Load the dataset
ds = load_dataset("sentence-transformers/stsb")

df = ds['train'].to_pandas()  

df.head()

data = {"sentence1": [], "sentence2": [], "label": []}
for i in range(len(df)):
    ##### TODO: Thực hành chuẩn bị dữ liệu#####
    # Thêm câu đầu tiên từ DataFrame vào danh sách "sentence1"
    # Thêm câu thứ hai từ DataFrame vào danh sách "sentence2"
    # Thêm điểm số từ DataFrame vào danh sách "label"
    
    #######################
    ### START CODE HERE ###
    #######################
    row = df.iloc[i]
    
    data["sentence1"].append(row['sentence1'])
    data["sentence2"].append(row['sentence2'])
    data["label"].append(row['score'])
    ##### End TODO #####
    
dataset = Dataset.from_dict(data)
dataset.save_to_disk("embedding_pair_dataset")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/471k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/108k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5749 [00:00<?, ? examples/s]

### **Bước 3: Tải Model, Tokenizer**

Tải base model Qwen/Qwen3-0.5B và tokenizer tương ứng từ Hugging Face Hub.

In [5]:
# Load and Prepare the Base Model
from transformers import AutoTokenizer, AutoModel
from peft import get_peft_model, LoraConfig, TaskType

base_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-Embedding-0.6B")
base_model = AutoModel.from_pretrained("Qwen/Qwen3-Embedding-0.6B")

2025-11-29 17:22:40.037161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764436960.198959      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764436960.246109      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

### **Bước 4: Khởi tạo và áp dụng cấu hình LoRA**

Sử dụng LoraConfig từ thư viện peft để định nghĩa cấu hình cho LoRA.

Chọn các tham số quan trọng:
- r: Thứ hạng (rank) của ma trận cập nhật (ví dụ: 8, 16, 32).
- lora_alpha: Hệ số co giãn (scaling factor).
- target_modules: Các lớp trong mô hình mà bạn muốn áp dụng LoRA (ví dụ: ["q_proj", "k_proj", "v_proj", "o_proj"]).
- lora_dropout: Tỷ lệ dropout trong các lớp LoRA.
- bias: Loại bias cần huấn luyện (thường là "none" hoặc "all").

Sử dụng hàm get_peft_model để áp dụng cấu hình LoRA vào mô hình nền tảng đã tải. In ra số lượng tham số có thể huấn luyện để thấy sự khác biệt


In [6]:
# Configure LoRA with target_modules
peft_config = LoraConfig(
    ##### TODO: Thực hành tạo cấu hình LoRA#####
    # Các tham số yêu cầu:
    # Thiết lập rank (thứ hạng) cho ma trận LoRA
    # Đặt hệ số co giãn
    # Cấu hình tỷ lệ dropout cho các lớp LoRA để chống overfitting.
    # Chọn không huấn luyện tham số bias để giữ cho việc fine-tuning hiệu quả nhất.
    # Chỉ định tác vụ là trích xuất đặc trưng (embedding) vì chúng ta cần output là vector.
    # Chọn các lớp mục tiêu để áp dụng LoRA
    

    #######################
    ### START CODE HERE ###
    #######################
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
    ##### End TODO #####
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()

trainable params: 4,587,520 || all params: 600,364,032 || trainable%: 0.7641


### **Bước 5: Xây dựng Vòng lặp Huấn luyện Tùy chỉnh (Custom Training Loop)**

Thay vì sử dụng Trainer của Hugging Face (như trong bài thực hành), chúng ta có thể xây dựng một vòng lặp huấn luyện tùy chỉnh theo một cách khác bằng PyTorch. Cách tiếp cận này mang lại sự linh hoạt cao hơn, cho phép chúng ta kiểm soát chi tiết từng bước của quá trình huấn luyện.

Để làm điều này, chúng ta cần chuẩn bị 3 thành phần chính:

- nn.Module (EmbeddingTrainer): Một lớp tùy chỉnh để định nghĩa cách mô hình xử lý dữ liệu đầu vào và tính toán loss.
- collate_fn: Một hàm để xử lý và đóng gói dữ liệu thành từng batch.
- DataLoader: Trình tải dữ liệu của PyTorch để tạo ra các batch một cách hiệu quả.


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmbeddingTrainer(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
    
    def forward(self, input_ids1, attn1, input_ids2, attn2, labels):
        ##### TODO: Thực hành #####
        # Yêu cầu:
        # Nhận vào hai câu đã được tokenize (input_ids1, input_ids2) và nhãn (labels).
        # Đưa từng câu qua base_model để lấy last_hidden_state.
        # Trích xuất embedding của câu bằng cách lấy vector của token [CLS] (ở vị trí đầu tiên [:, 0, :]).
        # Tính toán độ tương đồng cosine giữa hai embedding.
        # Tính toán loss (ví dụ: Mean Squared Error - MSE Loss) giữa độ tương đồng cosine dự đoán và nhãn thực tế.

        #######################
        ### START CODE HERE ###
        #######################
        out1 = self.base_model(input_ids=input_ids1, attention_mask=attn1)
        emb1 = out1.last_hidden_state[:, 0, :] 

        out2 = self.base_model(input_ids=input_ids2, attention_mask=attn2)
        emb2 = out2.last_hidden_state[:, 0, :]

        scores = F.cosine_similarity(emb1, emb2)

        loss = nn.MSELoss()(scores, labels)
        ##### End TODO #####
        
        return loss

def collate_fn(batch):
    ##### TODO: Thực hành #####

    # Yêu cầu:
    # Tách các câu và nhãn từ batch.
    # Sử dụng tokenizer để mã hóa các câu, đồng thời thêm padding để chúng có cùng độ dài.
    # Chuyển tất cả các tensor sang thiết bị tính toán (GPU/CPU).

    #######################
    ### START CODE HERE ###
    #######################
    sent1_list = [item['sentence1'] for item in batch]
    sent2_list = [item['sentence2'] for item in batch]
    labels_list = [item['label'] for item in batch]

    tok1 = base_tokenizer(
        sent1_list, 
        padding=True, 
        truncation=True, 
        max_length=128, 
        return_tensors="pt"
    )
    
    tok2 = base_tokenizer(
        sent2_list, 
        padding=True, 
        truncation=True, 
        max_length=128, 
        return_tensors="pt"
    )
    
    labels = torch.tensor(labels_list, dtype=torch.float32)
    ##### End TODO #####
    return (
        tok1['input_ids'].to(device),
        tok1['attention_mask'].to(device),
        tok2['input_ids'].to(device),
        tok2['attention_mask'].to(device),
        labels.to(device)
    )

from torch.utils.data import DataLoader
train_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

### **Bước 6: Viết Vòng lặp Huấn luyện và Tối ưu hóa**

Sau khi đã có DataLoader và EmbeddingTrainer, chúng ta sẽ viết vòng lặp huấn luyện chính. Đây là nơi quá trình học của mô hình thực sự diễn ra.

1. Khởi tạo Model và Optimizer
- EmbeddingTrainer: Chúng ta tạo một thực thể của lớp EmbeddingTrainer đã định nghĩa ở bước trước, truyền mô hình PEFT vào và chuyển toàn bộ sang thiết bị tính toán (device).
- Optimizer: Chúng ta chọn một trình tối ưu hóa. AdamW là một lựa chọn phổ biến và hiệu quả cho các mô hình Transformer. Quan trọng là chúng ta chỉ truyền trainer_model.parameters() vào optimizer. Nhờ có PEFT, hàm này sẽ chỉ trả về các tham số có thể huấn luyện (tức là các trọng số của LoRA), giúp quá trình tối ưu hóa cực kỳ hiệu quả.
2. Vòng lặp Huấn luyện
Vòng lặp huấn luyện bao gồm hai cấp:

- Vòng lặp ngoài (Epoch Loop): Lặp qua toàn bộ tập dữ liệu nhiều lần. Mỗi lần lặp được gọi là một epoch.
- Vòng lặp trong (Batch Loop): Lặp qua từng batch dữ liệu được cung cấp bởi DataLoader.

In [8]:
# Training
trainer_model = EmbeddingTrainer(model).to(device)
optimizer = torch.optim.AdamW(trainer_model.parameters(), lr=5e-5)

trainer_model.train()
for epoch in range(2):
    for batch in train_loader:
        #######################
        ### START CODE HERE ###
        #######################
        optimizer.zero_grad()
        input_ids1, attn1, input_ids2, attn2, labels = batch
        loss = trainer_model(input_ids1, attn1, input_ids2, attn2, labels)
        loss.backward()
        optimizer.step()
        
    print(f"[Epoch {epoch+1}] Loss: {loss.item():.4f}")

[Epoch 1] Loss: 0.0600
[Epoch 2] Loss: 0.0005


### **Bước 7:** Lưu model

In [9]:
# Save the model
model.save_pretrained("qwen3-lora-embedding-model")
base_tokenizer.save_pretrained("qwen3-lora-embedding-model")

('qwen3-lora-embedding-model/tokenizer_config.json',
 'qwen3-lora-embedding-model/special_tokens_map.json',
 'qwen3-lora-embedding-model/chat_template.jinja',
 'qwen3-lora-embedding-model/vocab.json',
 'qwen3-lora-embedding-model/merges.txt',
 'qwen3-lora-embedding-model/added_tokens.json',
 'qwen3-lora-embedding-model/tokenizer.json')